In [ ]:
import numpy as np
import sympy as sp
from sympy import *
from IPython.display import display, Math

## Szenari
Zur Beurteilung der betrieblichen Sicherheit eines Bauwerkes bei den durchgeführten Montagearbeiten werden Höhenmessungen (m) mit einem Nivellier über mehrere Tage durchgeführt.


In [ ]:
# Designmatrix A (hier: Beizahl y-Achsenabschnitt jeweils = 1, Beobachtungen / Meßwerte = x-Werte)
A = Matrix([
    [121.405, 1.],
    [121.404, 1.],
    [121.403, 1.],
    [121.409, 1.],
    [121.411, 1.],
    [121.403, 1.],
    [121.406, 1.],
    [121.407, 1.],
    [121.410, 1.],
    [121.412, 1.],
    [121.416, 1.],
    [121.412, 1.]
])

# b-Vektor (hier: Zeitangaben = y-Werte)
b = Matrix([
    [202.17],
    [204.04],
    [209.98],
    [213.52],
    [214.89],
    [217.01],
    [220.33],
    [225.06],
    [227.88],
    [233.01],
    [234.22],
    [240.09]
])

# Option 1: Matrix A transponieren, Darstellung in Ausgangsform mit Exponent "T" zur Kennzeichnung
A.T

# LaTeX-Darstellung der Matrizen mit Beschriftungen
latex_code = r"""
\text{Designmatrix A:} \\ A = %s \\
\text{B-Vektor:} \\ b = %s \\
\text{Transponierte Designmatrix A\_T:} \\ A\_T = %s
""" % (sp.latex(A), sp.latex(b), sp.latex(A.T))

# Anzeige der kombinierten LaTeX-Darstellung
display(Math(latex_code))

<IPython.core.display.Math object>

In [ ]:
# Genauigkeiten = Standardabweichungen für jeden einzelnen Messwert.

# 1.) gemessene Höhen außerhalb Design-Matrix (sympy-Matrix)
hoehen = Matrix([
    [121.405],
    [121.404],
    [121.403],
    [121.409],
    [121.411],
    [121.403],
    [121.406],
    [121.407],
    [121.410],
    [121.412],
    [121.416],
    [121.412]
])

# 2.) gemessene Höhen außerhalb Design-Matrix (numpy-Array)
hoehen_2 = np.array([
    [121.405, 121.404, 121.403, 121.409, 121.411, 121.403, 121.406, 121.407, 121.410, 121.412, 121.416, 121.412]
])

# Generieren der Gewichtsmatrix P aus den "mitgelieferten" Genauigkeiten (= Standardabweichungen)
# Gegebene Genauigkeiten (Standardabweichungen)
genauigkeiten = sp.Matrix([
    [0.005],
    [0.003],
    [0.003],
    [0.005],
    [0.005],
    [0.003],
    [0.003],
    [0.003],
    [0.003],
    [0.004],
    [0.005],
    [0.003]
])

In [18]:
# Berechnung der Varianzen (Die Zeile nimmt jede Standardabweichung in der Matrix genauigkeiten, quadriert sie und speichert das Ergebnis in der Matrix varianzen.)
varianzen = genauigkeiten.applyfunc(lambda x: x**2)
display(Math(latex(symbols('x')**2)))
display(Math(sp.latex(varianzen)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [17]:
# Berechnung der Kehrwerte der Varianzen (Gewichte)
gewichte = varianzen.applyfunc(lambda x: 1/x)
display(Math(latex(1/symbols('x'))))
display(Math(sp.latex(gewichte)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [15]:
# Extrahieren der Diagonalelemente in eine Liste
diagonale_gewichte = [float(g[0]) for g in gewichte.tolist()]
print(diagonale_gewichte)
#display(Math(sp.latex(diagonale_gewichte)))

[40000.0, 111111.11111111111, 111111.11111111111, 40000.0, 40000.0, 111111.11111111111, 111111.11111111111, 111111.11111111111, 111111.11111111111, 62500.0, 40000.0, 111111.11111111111]


In [20]:

# Berechnung des Minimums und Maximums der Diagonalelemente (Gewichte)
P_temp_min = min(diagonale_gewichte)
P_temp_max = max(diagonale_gewichte)
print(f"Minimum value of diagonal elements: {P_temp_min:.6f}")
print(f"Maximum value of diagonal elements: {P_temp_max:.6f}")

Minimum value of diagonal elements: 40000.000000
Maximum value of diagonal elements: 111111.111111


In [21]:
# Normalisierung der Diagonalelemente: (P - Min) / (Max - Min)
diagonale_gewichte_norm = [(g - P_temp_min) / (P_temp_max - P_temp_min) for g in diagonale_gewichte]
print(diagonale_gewichte_norm)

[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.31640625, 0.0, 1.0]


In [22]:
# Erstellung der Diagonalmatrix mit den normalisierten Diagonalelementen
P = sp.diag(*diagonale_gewichte_norm)
display(Math(sp.latex(P)))

<IPython.core.display.Math object>

In [23]:

# Normalgleichungsmatrix OHNE Gewichte
N = A.T * A 

display(Math(sp.latex(N)))

<IPython.core.display.Math object>

In [24]:
# Normalgleichungsmatrix MIT Gewichten
N_P = A.T * P * A 
display(Math(sp.latex(N_P)))

<IPython.core.display.Math object>

In [25]:
# Kofaktormatrix Q aus der invertierten Matrix N_P ( = (A_T * P * A).inv() )
Q = N_P.inv()
display(Math(sp.latex(Q)))

<IPython.core.display.Math object>

Wir haben nun:
- Die Designmatrix **A**
- die transponierte Designmatrix **A_T**
- den Vektor **b**
- die Gewichtsmatrix **P**
- die Normalgleichungsmatrix **N (bzw. N_P)**
- die Kofaktormatrix **Q**
- den Vektor der geschätzten Parameter **x**
- den Vektor der Modellwerte **l**
- Vektor der Verbesserungen **v**.

In [29]:
# Ermitteln der Parameter (Steigung, y-Achsenabschnitt) für die Geradengleichung
ATPA = (A.T * P * A)
display(Math(sp.latex(ATPA)))

<IPython.core.display.Math object>

In [27]:
ATPA_inv = ATPA.inv()
display(Math(sp.latex(ATPA_inv)))

<IPython.core.display.Math object>

In [26]:
x = ATPA_inv * (A.T * P * b)
display(Math(sp.latex(x)))

<IPython.core.display.Math object>

In [ ]:
# Vektor der geschätzten Modellwerte (l)
l = A * x

# Vektor der Verbesserungen (v)
v = b - l

# Probe: Geradengleichung als Funktion definieren, bekannten x-Wert wählen: y-Wert passend? (Achtung: Verbesserung berücksichtigt, 
# deshalb ist hier nicht mehr der y-Ausgangswert aus der Tabelle zu erwarten!)
def lineare_funktion(x, m, n):
    return m * x + n

# Steigung (m) und y-Achsenabschnitt (n)
m = 2972.81245517731  # Steigung
n = -360698.096679688  # y-Achsenabschnitt

# Beispiel für einen gegebenen x-Wert
x_wert = 121.412

# Berechnung des y-Wertes
y_wert = lineare_funktion(x_wert, m, n)

# Ausgabe des Ergebnisses
print(f"Für den x-Ausgangswert = {x_wert:.3f} beträgt der (verbesserte) y-Wert: {y_wert:.3f}")
# Display the formulas
display(Math(sp.latex(varianzen)))
display(Math(sp.latex(gewichte)))
display(Math(sp.latex(P)))
# Funktionsgleichung: f(x) = 2972.81245x - 360698.096679688